In [7]:
import matplotlib.pyplot as plt
import numpy as np
%config InlineBackend.figure_format = 'svg'

In [8]:
def generate_random_state(n):
    random_states = np.random.uniform(0, 1, size=(n, 6)) + 1j*np.random.uniform(0, 1, size=(n, 6))
    random_states = random_states/np.linalg.norm(random_states, axis=1).reshape(-1, 1)
    return random_states

def von_neumann_entropy(eigvals, dim=2):
    res = 0
    for eig in eigvals:
        if eig == 0:
            continue
        res += eig*np.log2(1/eig)

    return np.round(res, 3)

In [9]:
state = generate_random_state(1000)

In [10]:
def partial_traces(state, dimA, dimB):
    st = state.reshape(-1, 1)
    density_m = st@np.conj(st.T)
    # print("DENSITY MATRIX : ", density_m)
    
    density_A = np.zeros((dimA, dimA)).astype(np.complex128)
    density_B = np.zeros((dimB, dimB)).astype(np.complex128)

    for row in range(dimA):
        for col in range(dimA):
            density_A[row, col] = np.sum([density_m[dimB*row + i, dimB*col + i] for i in range(dimB)]) 

    for row in range(dimB):
        for col in range(dimB):
            density_B[row, col] = np.sum([density_m[row + i*dimB, col + i*dimB] for i in range(dimA)])

    return (density_A, density_B) 

def tensor_prodcut(state):
    n = len(state)
    res = np.zeros(n*n).astype(np.complex128)

    for idx, el in enumerate(state):
        res[n*idx: n*(idx+1)] = el*state

    return res

In [11]:
pa, pb = partial_traces(state[0], 2, 3)

In [12]:
state[0]

array([0.03992009+0.41268727j, 0.00461254+0.24675666j,
       0.24169901+0.11376415j, 0.44036787+0.03531053j,
       0.20486975+0.46861434j, 0.47798408+0.10302573j])

In [15]:
print(np.linalg.eigvalsh(pa))

[0.07435319 0.92564681]


In [16]:
print(np.linalg.eigvalsh(pb))

[7.32866149e-17 7.43531875e-02 9.25646813e-01]


In [17]:
# tensor_prodcut(state[0])

## Entanglement Entropy of qubit-qutrit pure states

Let us lay down some definitions:

Let $\mathcal{H}_A$, $\mathcal{H}_B$ be **Hilbert Spaces** where,

$$ \dim(\mathcal{H}_A) = 2, \quad \dim(\mathcal{H}_B) = 3 $$

Let $\ket{0}_A$, $\ket{1}_A$ be the orthogonal basis states of $\mathcal{H}_A$.
Similarly $\ket{0}_B$, $\ket{1}_B$ and $\ket{2}_B$ be the orthogonal states of $\mathcal{H}_B$.


We will now consider an arbitrary bipartite statevector,  $$\ket{\psi} = a\ket{0}_A \ket{0}_B + b\ket{0}_A\ket{1}_B + c\ket{0}_A\ket{2}_B + d\ket{1}_A \ket{0}_B + e \ket{1}_A \ket{1}_B + f \ket{1}_A \ket{2}_B $$
$$ \ket{\psi} \in \mathcal{H}_A \otimes \mathcal{H}_B $$
Correspondingly, 

$$  \rho = \ket{\psi} \bra{\psi}, \quad \rho \in \mathcal{D}(\mathcal{H}_A \otimes \mathcal{H}_B) $$

The ***Entanglement Entropy*** of the state $\ket{\psi}$ is defined as:

$$ \mathcal{E}(\rho) = \mathcal{S}(\rho_A) = \mathcal{S}(\rho_B) $$

where $\mathcal{S}$ is the *von Neumann Entropy* and $\rho_A$, $\rho_B$ are the reduced subsystems.

From the **Schmidt Decomposition** we can conclude that ranks of the matrices $\rho_A$ and $\rho_B$ and have the same eigenvalues.

From further simplification we find the eigenvalues to be,

$\lambda_\pm  = \dfrac{1 \pm \sqrt{1-4(|ae-bd|^2 + |af-cd|^2 + |bf-ce|^2)}}{2}$

Let us call the parameter to be estimated as $\beta$, then

$$ \lambda_\pm = \dfrac{1 \pm \sqrt{1 - 4\beta}}{2}, \quad 0\leq \beta \leq \frac{1}{4} $$


### UNITARY OPERATOR DEFINITION


States of interest : 

$$ ae : \left(\ket{0}_A \ket{0}_B \ket{1}_A \ket{1}_B \quad \ket{1}_A \ket{1}_B \ket{0}_A \ket{0}_B \right), \quad bd : (\ket{0}_A \ket{1}_B \ket{1}_A \ket{0}_B \quad \ket{1}_A \ket{0}_B \ket{0}_A \ket{1}_B) \implies |ae-bd|^2$$
$$ af : (\ket{0}_A \ket{0}_B \ket{1}_A \ket{2}_B \quad \ket{1}_A \ket{2}_B \ket{0}_A \ket{0}_B), \quad cd : (\ket{0}_A \ket{2}_B \ket{1}_A \ket{0}_B \quad \ket{1}_A \ket{0}_B \ket{0}_A \ket{2}_B) \implies |af-cd|^2$$
$$ bf : (\ket{0}_A \ket{1}_B \ket{1}_A \ket{2}_B \quad \ket{1}_A \ket{2}_B \ket{0}_A \ket{1}_B), \quad ce : (\ket{0}_A \ket{2}_B \ket{1}_A \ket{1}_B) \quad \ket{1}_A \ket{1}_B \ket{0}_A \ket{2}_B) \implies |bf-ce|^2$$

### LOCC 

We will now define a LOCC Protcol for the by measuring the qutrit subsystem and apply local unitary operations depending upon the measured outcome

We propose a of local 2-qubit unitary, that would be used based upon measurement outcomes.

$$ U (\ket{01}) \rightarrow \frac{1}{\sqrt{2}} ( \ket{01} - \ket{10}) $$

$$ U (\ket{10}) \rightarrow \frac{1}{\sqrt{2}} ( \ket{10} + \ket{01}) $$

We measure Bob's pairs of qubits $q_{B1}$ and $q_{B2}$.

If $q_{B1} + q_{B2} = 1$, or $(q_{B1}, q_{B2}) = (0, 2)$ or $(q_{B1}, q_{B2}) = (2, 0)$ or $q_{B1} + q_{B2} = 3$, we perform the local unitary operation on Alice's pair of qubits

Finally, we measure the relative frequencies of observing either of the following states:

$$ \ket{1}_A \ket{1}_B \ket{0}_A \ket{0}_B, \quad \ket{1}_A \ket{0}_B \ket{0}_A \ket{1}_B \implies |ae-bd|^2$$
$$ \ket{1}_A \ket{2}_B \ket{0}_A \ket{0}_B, \quad \ket{1}_A \ket{0}_B \ket{0}_A \ket{2}_B \implies |af-cd|^2$$
$$ \ket{1}_A \ket{2}_B \ket{0}_A \ket{1}_B, \quad \ket{1}_A \ket{1}_B \ket{0}_A \ket{2}_B \implies |bf-ce|^2$$

By doing so, we obtain the estimate $\hat{\beta}$
